<a href="https://colab.research.google.com/github/Heba2424/DEPI_Projects/blob/main/Dense_Retrieval_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch --quiet
!pip install PyMuPDF --quiet
!pip install transformers torch numpy --quiet
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 118.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4

In [2]:
import requests
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Fetch and clean the text from the book
url = "https://www.gutenberg.org/files/1661/1661-0.txt"
response = requests.get(url)
text = response.text

# Print a portion to inspect content
print(text[:10000])  # Print first 10,000 characters for review

# Example of flexible extraction
start_index = text.find("CHAPTER I")  # Adjust based on inspection
end_index = text.find("End of Project Gutenberg's")  # Adjust based on inspection

if start_index == -1:
    start_index = 0
if end_index == -1:
    end_index = len(text)

text = text[start_index:end_index]

ï»¿The Project Gutenberg eBook of The Adventures of Sherlock Holmes,
by Arthur Conan Doyle

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: The Adventures of Sherlock Holmes

Author: Arthur Conan Doyle

Release Date: November 29, 2002 [eBook #1661]
[Most recently updated: October 10, 2023]

Language: English

Character set encoding: UTF-8

Produced by: an anonymous Project Gutenberg volunteer and Jose Menendez

*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK
HOLMES ***




The Adventures of Sherlock Holmes

by Arthur Conan Doyle


Contents

   I.     A Scandal in Bohemia

In [3]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [4]:
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding='max_length', max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state, inputs['attention_mask']

def mean_pooling(embeddings, attention_mask):
    token_embeddings = embeddings.squeeze()
    attention_mask = attention_mask.squeeze()
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size())
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask



In [5]:
# Split text into chunks
def split_text(text, chunk_size=500, overlap=100):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunk = text[i:i + chunk_size]
        chunks.append(chunk)
    return chunks

# Get embeddings for chunks
def get_text_embeddings(text_chunks, batch_size=8):
    embeddings = []
    for i in range(0, len(text_chunks), batch_size):
        batch_chunks = text_chunks[i:i + batch_size]
        batch_embeddings = []
        for chunk in batch_chunks:
            chunk_embeddings, attention_mask = get_bert_embeddings(chunk)
            chunk_embedding = mean_pooling(chunk_embeddings, attention_mask)
            # Ensure consistent shape
            chunk_embedding = chunk_embedding.cpu().numpy()
            print(f"Chunk shape: {chunk_embedding.shape}")  # Debug: Print shape of chunk embeddings
            batch_embeddings.append(chunk_embedding)
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

text_chunks = split_text(text)
text_embeddings = get_text_embeddings(text_chunks)

Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)
Chunk shape: (512,)


In [6]:
# Query embedding
query = "What is the central mystery in 'The Adventures of Sherlock Holmes'?"
query_embeddings, query_attention_mask = get_bert_embeddings(query)
query_embedding = mean_pooling(query_embeddings, query_attention_mask).cpu().numpy()

In [7]:
import numpy as np

def cosine_similarity(embedding1, embedding2):
    dot_product = np.dot(embedding1, embedding2)
    norm1 = np.linalg.norm(embedding1)
    norm2 = np.linalg.norm(embedding2)
    if norm1 == 0 or norm2 == 0:
        return 0
    return dot_product / (norm1 * norm2)



In [8]:
similarities = []
for i, text_embedding in enumerate(text_embeddings):
    similarity = cosine_similarity(query_embedding.flatten(), text_embedding.flatten())
    similarities.append((i, similarity))


In [9]:
# Sort by similarity and apply threshold
threshold = 0.40
sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
filtered_similarities = [(i, sim) for i, sim in sorted_similarities if sim >= threshold]


In [10]:
# Output the results
print(f"Number of matches above threshold: {len(filtered_similarities)}")
for idx, sim in filtered_similarities:
    print(f"Chunk Index: {idx}, Similarity: {sim}")


Number of matches above threshold: 19
Chunk Index: 1518, Similarity: 0.4900580942630768
Chunk Index: 557, Similarity: 0.43785205483436584
Chunk Index: 31, Similarity: 0.43441128730773926
Chunk Index: 1510, Similarity: 0.43053582310676575
Chunk Index: 1387, Similarity: 0.4203222095966339
Chunk Index: 1499, Similarity: 0.4145791828632355
Chunk Index: 42, Similarity: 0.41365060210227966
Chunk Index: 133, Similarity: 0.41360750794410706
Chunk Index: 1493, Similarity: 0.41224637627601624
Chunk Index: 1311, Similarity: 0.4089495837688446
Chunk Index: 1107, Similarity: 0.40748047828674316
Chunk Index: 1474, Similarity: 0.4067314565181732
Chunk Index: 182, Similarity: 0.40504416823387146
Chunk Index: 363, Similarity: 0.4040771722793579
Chunk Index: 1125, Similarity: 0.4033162593841553
Chunk Index: 1504, Similarity: 0.40314799547195435
Chunk Index: 1263, Similarity: 0.40155068039894104
Chunk Index: 845, Similarity: 0.4008632004261017
Chunk Index: 1424, Similarity: 0.4000116288661957


In [11]:
print("Query Embedding Shape:", query_embedding.shape)
print("Sample Text Embedding Shape:", text_embeddings[0].shape)


Query Embedding Shape: (512,)
Sample Text Embedding Shape: (512,)


In [12]:
# Sort by similarity and apply threshold
threshold = 0.40  # Adjusted threshold
sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
filtered_similarities = [(i, sim) for i, sim in sorted_similarities if sim >= threshold]

# Output the results
print(f"Number of matches above threshold: {len(filtered_similarities)}")

# Print the filtered chunks
for idx, sim in filtered_similarities:
    print(f"Chunk Index: {idx}, Similarity: {sim}")
    print(f"Chunk Text: {text_chunks[idx][:1000]}...")  # Print the first 1000 characters of the chunk for brevity

Number of matches above threshold: 19
Chunk Index: 1518, Similarity: 0.4900580942630768
Chunk Text:  search
facility: www.gutenberg.org.

This website includes information about Project Gutenbergâ¢,
including how to make donations to the Project Gutenberg Literary
Archive Foundation, how to help produce our new eBooks, and how to
subscribe to our email newsletter to hear about new eBooks.


...
Chunk Index: 557, Similarity: 0.43785205483436584
Chunk Text: m it. In the first place, we may start with a
strong presumption that Colonel Openshaw had some very strong reason
for leaving America. Men at his time of life do not change all their
habits and exchange willingly the charming climate of Florida for the
lonely life of an English provincial town. His extreme love of solitude
in England suggests the idea that he was in fear of someone or
something, so we may assume as a working hypothesis that it was fear of
someone or something which drove...
Chunk Index: 31, Similarity: 0.43441128730

In [13]:
import gradio as gr
import torch

def process_query(query):
    # Get embedding for query
    query_embeddings, query_attention_mask = get_bert_embeddings(query)
    query_embedding = mean_pooling(query_embeddings, query_attention_mask).cpu().numpy().flatten()

    # Compute similarity between query and each chunk
    similarities = []
    for i, text_embedding in enumerate(text_embeddings):
        similarity = cosine_similarity(query_embedding, text_embedding.flatten())
        similarities.append((i, similarity))

    # Sort by similarity and apply threshold
    threshold = 0.40
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    filtered_similarities = [(i, sim) for i, sim in sorted_similarities if sim >= threshold]

    # Format results
    results = []
    for idx, sim in filtered_similarities:
        results.append(f"Chunk Index: {idx}\nSimilarity: {sim:.2f}\nText: {text_chunks[idx]}")
    return "\n\n".join(results)

# Gradio interface
iface = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs="text",
    title="Text Similarity Search",
    description="Enter a query to find similar chunks of text from the provided book."
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9f90fdaab2a7d04c93.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
